In [ ]:
# %%
%pip install mlxtend

# %%
# import libraries
import numpy as np
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from mlxtend.frequent_patterns import association_rules, apriori # for mining frequent itemsets and association rule

import warnings
warnings.filterwarnings("ignore")

# %%
file_path = 'Basket Analysis NoDups.csv'

# Read the CSV file into a DataFrame
df = pd.read_csv(file_path)

df.columns

# %%
# counting the number of unique claims
print('The total number of unique claims is ', df['Family ID'].nunique())

# %%
print(df.dtypes)

# %%
# convert date_time column into the right format for easier extracting
df['date_column'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')

# %%
# extracting month and display full name of the month
df['month'] = df['date_column'].dt.strftime('%Y-%m')

# %%
# List of specific columns you want to keep
specific_columns = ['Family ID', 'LA', 'month', 'Crit-1',
       'Crit-2', 'Crit-3', 'Crit-4', 'Crit-5', 'Crit-6', 'Crit-7', 'Crit-8',
       'Crit-9', 'Crit-10']

# Create a DataFrame with specific columns
df_specific = pd.DataFrame({col: df[col] for col in specific_columns})
'''
# List of specific columns you want to keep
specific_columns2 = ['Family ID', 'LA', 'month', 'C 1.1',
       'C 1.2', 'C 1.3', 'C 1.4', 'C 2.1', 'C 2.2', 'C 2.3', 'C 3.1', 'C 3.2',
       'C 3.3', 'C 4.1', 'C 4.2', 'C 5.1', 'C 5.2', 'C 5.3', 'C 5.4', 'C 6.1',
       'C 6.2', 'C 6.3', 'C 6.4', 'C 6.5', 'C 7.1', 'C 7.2', 'C 7.3', 'C 8.1',
       'C 8.2', 'C 8.3', 'C 9.1', 'C 9.2', 'C 9.3', 'C 10.1', 'C 10.2',
       'C 10.3']

# Create a DataFrame with specific columns
df_specific2 = pd.DataFrame({col: df[col] for col in specific_columns2})
'''

# %%
# Unpivot the criteria columns
df_unpivoted = pd.melt(df_specific, id_vars=['Family ID', 'LA', 'month'], var_name='Criteria', value_name='Criteria_Value')

# Filter rows where Criteria_Value is 1
df_unpivoted = df_unpivoted[df_unpivoted['Criteria_Value'] == 1]

# Drop duplicate rows based on 'FamilyID' and 'Criteria'
df_no_dup = df_unpivoted.drop_duplicates(subset=['Family ID', 'Criteria'])

# Order by Family ID
df_no_dup.sort_values(by='Family ID', ascending=False, inplace=True)

# Reset index
df_no_dup.reset_index(drop=True, inplace=True)

'''''
# Unpivot the criteria columns
df_unpivoted2 = pd.melt(df_specific2, id_vars=['Family ID', 'LA', 'month'], var_name='C', value_name='Criteria_Value')

# Filter rows where Criteria_Value is 1
df_unpivoted2 = df_unpivoted2[df_unpivoted2['Criteria_Value'] == 1]

# Reset index
df_unpivoted2.reset_index(drop=True, inplace=True)


'''

# %%
df_no_dup

# %% [markdown]
# Criteria Distribution

# %%
# most common occuring criteria
top_items = pd.DataFrame(df_no_dup['Criteria'].value_counts(dropna=True, sort=True)).reset_index()
top_items.columns = ['item', 'count']
top_items['percentage'] = top_items['count'].apply(lambda x: x/top_items['count'].sum())
top_items = top_items.head(10)
top_items.head(10)

# %%
# create bar plot showing criteria distribution
plt.figure(figsize=(12,5))
sns.barplot(x = 'item', y = 'count', data = top_items, palette = 'viridis')
plt.xlabel('Criteria', size = 12)
plt.xticks(rotation=45)
plt.ylabel('Number of Claims', size = 12)
plt.title('Criteria Distribution', size = 15)


# %% [markdown]
# Number of criteria per Claim

# %%
# create summary table of number of transactions by their number of items
# count the number of items of each transaction
items_num = df_no_dup.groupby('Family ID', as_index=False)['Criteria'].count()
# pivot table to display the distribution of transactions by the number of items
items_num = items_num.pivot_table(index='Criteria', values='Family ID', aggfunc='count')
# calculate percentage
items_num['percentage'] = items_num['Family ID'].apply(lambda x: x/items_num['Family ID'].sum())
items_num

# %% [markdown]
# Apply Apriori Algorithm to implement Market Basket Analysis

# %%
df_no_dup

# %%
# select only required variables for modelling
transactions = df_no_dup.groupby(['Family ID', 'Criteria'])['Criteria'].count().reset_index(name ='Count')
transactions.head()

# %%
# first create a mxn matrice where m=transaction and n=items
# each row represents whether the items was in a specific transaction or not (>=1 returns True (1), 0 returns 0)
my_basket = transactions.pivot_table(index='Family ID', columns='Criteria', values='Count', aggfunc='any').fillna(0)

my_basket.head()

# %%
# create frequent items df with itemsets and support columns by using `apriori` function
frequent_items = apriori(my_basket, min_support = 0.01, use_colnames = True)
frequent_items

# %%
# create the rules from frequent itemset generated above with min lift = 1.2
rules = association_rules(frequent_items, metric = "lift", min_threshold = 1.2)
rules.sort_values('confidence', ascending = False, inplace = True)
rules.reset_index(drop=True, inplace = True)
rules

# %% [markdown]
# Parallel coordinates plot

# %%
# Function to convert rules to coordinates.
def rules_to_coordinates(rules):
    rules['antecedent'] = rules['antecedents'].apply(lambda antecedent: list(antecedent)[0])
    rules['consequent'] = rules['consequents'].apply(lambda consequent: list(consequent)[0])
    rules['rule'] = rules.index
    return rules[['antecedent','consequent','rule']]

# import sub lib to plot parallel coordinates
from pandas.plotting import parallel_coordinates

# Convert rules into coordinates suitable for use in a parallel coordinates plot
coords = rules_to_coordinates(rules)

# Generate parallel coordinates plot
plt.figure(figsize=(5,5))
parallel_coordinates(coords, 'rule')
plt.legend([])
plt.grid(ls='--', lw='0.5')
plt.title('Parallel coordinates', size = 15)
plt.show()

# %%
# convert antecedents and consequents into strings
rules['antecedents'] = rules['antecedents'].apply(lambda a: ','.join(list(a)))
rules['consequents'] = rules['consequents'].apply(lambda a: ','.join(list(a)))

# transform antecedent, consequent, and support columns into matrix
support_table_lift = rules.pivot(index='consequents', columns='antecedents', values='lift')

# generate a heatmap with annotations 
plt.figure(figsize=(10,6))
sns.heatmap(support_table_lift, annot = True, cbar = True, cmap="RdPu")
plt.suptitle('Itemsets Lift', size = 15)
plt.title('How many times the antecedents and the consequents occur together more often than random?\n', size=10)
plt.show()

# %% [markdown]
# Itemsets Confidence Heatmap

# %%
# transform antecedent, consequent, and support columns into matrix
rules_confidence = rules[rules['confidence']>=0.2] # select min lift=1.2
support_table_conf = rules_confidence.pivot(index='consequents', columns='antecedents', values='confidence')

# generate a heatmap with annotations
plt.figure(figsize=(10,6))
sns.heatmap(support_table_conf, annot = True, cbar = True, cmap="BuPu")
plt.suptitle('Itemsets Confidence', size = 15)
plt.title('How often the consequents is purchased given that the antecedents was purchased?\n', size = 10)
plt.show()


